# Вхождение точек в полигон:

### Шаг 1. Работа с shape - файлом:

In [69]:
import pandas as pd
pd.set_option('display.max_columns', 100)
from simpledbf import Dbf5
import fiona
import matplotlib.path as mplPath
import numpy as np
import re
from collections import OrderedDict
from pandas.io.json import json_normalize

<u/>Открываем файл и вытаскиваем координаты любого полигона:

Для этого напишем функцию. На вход она будет принимать уникальный код полигона, а на выходе мы будем получать координаты полигона:

In [70]:
'''В этой ячейке содержится функция для поиска вхождения точки в полигоны межселенных территорий. Оставлю пока ее здесь'''
# def coord_of_poly(uniq_code):
#     shape = fiona.open("C:/Users/User/PycharmProjects/scientificProject/data/Inter-settlement_territories.shp")
#     first = shape[1]
#     dbf = Dbf5("C:/Users/User/PycharmProjects/scientificProject/data/Inter-settlement_territories.dbf")
#     df = dbf.to_dataframe() 
#     for i in range(0, len(shape)):
#         if uniq_code in shape[i]['properties']['Код']:
#             coord = shape[i]['geometry']['coordinates']    # вытаскиваем координаты введенного полигона
#             a = shape[i]['properties']                     # выводим название выбранного полигона
#             a = OrderedDict(sorted(a.items(), key=lambda t: len(t[0])))
#             a = a.popitem() 
#             a = a[1]
# #     print()
# #     print("Название полигона: {}".format(a))
# #     print()
# #     print("Координаты полигона:")
#     return coord, a
# # coordinates_polygone = input('Введите уникальный код межселенной территории: ')
# # coordinates_0 = coord_of_poly(coordinates_polygone)
# # coordinates_0

'В этой ячейке содержится функция для поиска вхождения точки в полигоны межселенных территорий. Оставлю пока ее здесь'

Видоизменим функцию для вывода координат, т.к. нам нужны районы (т.е. файл boundary-polygon.shp):

In [71]:
def coord_of_reg(uniq_code):
    shape = fiona.open("C:/Users/User/Documents/boundary_file/RU-5layers-big3-20201030-ru-shape/boundary-polygon.shp")
    first = shape[1]
    for i in range(0, len(shape)):
        if shape[i]['properties']['OSM_ID'] == uniq_code:
            coord = shape[i]['geometry']['coordinates']    # вытаскиваем координаты введенного полигона
            a = shape[i]['properties']                     # выводим название выбранного полигона
            a = OrderedDict(sorted(a.items(), key=lambda t: len(t[0])))
            a = a.popitem(last=False) 
            a = a[1]
#     print()
#     print("Название полигона: {}".format(a))
#     print()
#     print("Координаты полигона:")
    return coord, a

Теперь преобразуем файл .shp в датафрейм:

In [73]:
shape = fiona.open("C:/Users/User/Documents/boundary_file/RU-5layers-big3-20201030-ru-shape/boundary-polygon.shp")
s = pd.DataFrame(shape)   # загоняем в датафрейм
s = s.reset_index()

In [74]:
n = json_normalize(data = s['properties'], meta=['id'])    # парсим содержимое словаря "properties" на разные столбцы
n = n.reindex(sorted(n.columns, reverse=True, key=len), axis=1)
n = n.reset_index()

<ipython-input-74-4ea7e8808b26>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  n = json_normalize(data = s['properties'], meta=['id'])    # парсим содержимое словаря "properties" на разные столбцы


In [75]:
m = json_normalize(data = s['geometry'], meta=['id'])    # парсим содержимое словаря "geometry" на разные столбцы
m = m.reindex(sorted(m.columns, reverse=True, key=len), axis=1)
m = m.reset_index()

<ipython-input-75-3ca06b5d601f>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  m = json_normalize(data = s['geometry'], meta=['id'])    # парсим содержимое словаря "geometry" на разные столбцы


In [101]:
newdata = s.merge(n, on = 'index', how='left')          # объединяем всё, что у нас получилось
newdata = newdata.merge(m, on = 'index', how='left')
newdata = newdata.drop(['index', 'type_x', 'id', 'properties', 'geometry'], axis=1)    # удаляем ненужные столбцы (распарсенные)
newdata

,ADMIN_L10D,ADMIN_LVL,ADMIN_L1D,ADMIN_L2D,ADMIN_L3D,ADMIN_L4D,ADMIN_L5D,ADMIN_L6D,ADMIN_L7D,ADMIN_L8D,ADMIN_L9D,ADMIN_L10,OSM_TYPE,ADMIN_L1,ADMIN_L2,ADMIN_L3,ADMIN_L4,ADMIN_L5,ADMIN_L6,ADMIN_L7,ADMIN_L8,ADMIN_L9,NAME_EN,NAME_RU,OSM_ID,oktmo,NAME,coordinates,type_y
0,None,9,None,60189.0,1221148.0,140295.0,NaN,1591241.0,NaN,NaN,NaN,None,relation,None,Россия,Сибирский федеральный округ,Томская область,None,городской округ Томск,None,None,None,None,None,269721,None,Ленинский район,"[[(84.9110562, 56.5705856), (84.9132019, 56.57...",Polygon
1,None,6,None,60189.0,1059500.0,108082.0,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,Южный федеральный округ,Краснодарский край,None,None,None,None,None,Krasnodar Municipality,None,269701,03701000,городской округ Краснодар,"[[(38.8105254, 45.2131433), (38.82135, 45.2051...",Polygon
2,None,6,None,60189.0,1221148.0,140295.0,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,Сибирский федеральный округ,Томская область,None,None,None,None,None,Pervomay region,Первомайский район,270287,69648000,Первомайский район,"[[(87.9210015, 58.2283232), (87.8446271, 58.11...",Polygon
3,None,5,None,60189.0,1029256.0,102269.0,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,Центральный федеральный округ,Москва,None,None,None,None,None,Western Administrative Okrug,Западный административный округ,226149,None,Западный административный округ,"[[[(37.3936145, 55.6947794), (37.3933397, 55.6...",MultiPolygon
4,None,9,None,60189.0,1216601.0,103906.0,NaN,331414.0,NaN,NaN,NaN,None,relation,None,Россия,Северо-Западный федеральный округ,Калининградская область,None,городской округ Калининград,None,None,None,None,Московский район,331413,None,Московский район,"[[(20.3051808, 54.6401712), (20.3048011, 54.64...",Polygon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25044,None,4,None,60189.0,1216601.0,NaN,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,Северо-Западный федеральный округ,None,None,None,None,None,None,Pskov Oblast,Псковская область,155262,58000000,Псковская область,"[[(27.736421428576882, 58.977562857144186), (2...",Polygon
25045,None,4,None,60189.0,1221148.0,NaN,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,Сибирский федеральный округ,None,None,None,None,None,None,Tuva Republic,Республика Тыва,145195,93000000,Республика Тыва,"[[(89.60883370417781, 49.91509098543708), (89....",Polygon
25046,None,3,None,60189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,None,None,None,None,None,None,None,Northwestern Federal District,Северо-Западный федеральный округ,1216601,None,Северо-Западный федеральный округ,"[[[(64.93639726375713, 69.57268782637112), (64...",MultiPolygon
25047,None,3,None,60189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,None,None,None,None,None,None,None,Siberian Federal District,Сибирский федеральный округ,1221148,None,Сибирский федеральный округ,"[[[(86.5722532961104, 49.718584921545336), (86...",MultiPolygon


In [102]:
newdata.to_excel('boundary файл в формате.xlsx')       # загоняем в эксель

In [107]:
shape = fiona.open("C:/Users/User/Documents/boundary_file/RU-5layers-big3-20201030-ru-shape/boundary-polygon.shp")
first = shape[22487]
first

{'type': 'Feature',
 'id': '22487',
 'properties': OrderedDict([('NAME', 'Белгородский район'),
              ('NAME_EN', 'Belgorodsky District'),
              ('NAME_RU', 'Белгородский район'),
              ('ADMIN_LVL', '6'),
              ('OSM_TYPE', 'relation'),
              ('OSM_ID', 1645362),
              ('ADMIN_L1D', None),
              ('ADMIN_L1', None),
              ('ADMIN_L2D', 60189),
              ('ADMIN_L2', 'Россия'),
              ('ADMIN_L3D', 1029256),
              ('ADMIN_L3', 'Центральный федеральный округ'),
              ('ADMIN_L4D', 83184),
              ('ADMIN_L4', 'Белгородская область'),
              ('ADMIN_L5D', None),
              ('ADMIN_L5', None),
              ('ADMIN_L6D', None),
              ('ADMIN_L6', None),
              ('ADMIN_L7D', None),
              ('ADMIN_L7', None),
              ('ADMIN_L8D', None),
              ('ADMIN_L8', None),
              ('ADMIN_L9D', None),
              ('ADMIN_L9', None),
              ('ADMI

In [106]:
newdata[newdata['NAME'] == 'Белгородский район']

,ADMIN_L10D,ADMIN_LVL,ADMIN_L1D,ADMIN_L2D,ADMIN_L3D,ADMIN_L4D,ADMIN_L5D,ADMIN_L6D,ADMIN_L7D,ADMIN_L8D,ADMIN_L9D,ADMIN_L10,OSM_TYPE,ADMIN_L1,ADMIN_L2,ADMIN_L3,ADMIN_L4,ADMIN_L5,ADMIN_L6,ADMIN_L7,ADMIN_L8,ADMIN_L9,NAME_EN,NAME_RU,OSM_ID,oktmo,NAME,coordinates,type_y
22487,None,6,None,60189.0,1029256.0,83184.0,NaN,NaN,NaN,NaN,NaN,None,relation,None,Россия,Центральный федеральный округ,Белгородская область,None,None,None,None,None,Belgorodsky District,Белгородский район,1645362,14610000,Белгородский район,"[[(36.0918, 50.44544), (36.08665, 50.44836), (...",Polygon


In [113]:
o = newdata[newdata['NAME'] == 'Белгородский район']['coordinates'].tolist()
o

[[[(36.0918, 50.44544),
   (36.08665, 50.44836),
   (36.0907742, 50.4488776),
   (36.1102185, 50.4526667),
   (36.1181806, 50.4576826),
   (36.1314228, 50.4638184),
   (36.1312552, 50.4644053),
   (36.1423183, 50.4696867),
   (36.1494174, 50.4740133),
   (36.1644446, 50.4832344),
   (36.1755077, 50.4872873),
   (36.1702276, 50.4984309),
   (36.1679646, 50.5037086),
   (36.1470956, 50.5015763),
   (36.1413964, 50.5001369),
   (36.1413577, 50.5003253),
   (36.1397401, 50.5081978),
   (36.1394687, 50.5095186),
   (36.1367029, 50.5161806),
   (36.1366823, 50.5163528),
   (36.1354458, 50.5266781),
   (36.1289923, 50.5271576),
   (36.128657, 50.5291822),
   (36.1216168, 50.5300879),
   (36.122455, 50.5388778),
   (36.1306954, 50.5430559),
   (36.1313507, 50.5429685),
   (36.1319296, 50.5429022),
   (36.1320763, 50.5429088),
   (36.1320725, 50.5429333),
   (36.1320964, 50.5429754),
   (36.1321147, 50.5430565),
   (36.1321367, 50.5431429),
   (36.1320606, 50.5431364),
   (36.1316636, 50.543182

Перейдем к файлу, из которого мы будем брать наши id для вышеуказанной функции:

In [4]:
oktmo = pd.read_excel('C:/Users/User/Documents/boundary_file/Справочник ОКТМО для районов.xlsx')
oktmo

,Код,Тип,Наименование,Родитель,Дополнительные данные,Том,Численность,Наименование.1,Код.1,Комментарий,Актуальность,Источник,id территории,0.988909426987061,Unnamed: 14
0,RF,Страна,Российская Федерация,NaN,NaN,NaN,146748590,NaN,NaN,NaN,1,Консультант,60189.0,NaN,NaN
1,FO1,Федеральный округ,ЦЕНТРАЛЬНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,RF,NaN,NaN,39433556,NaN,NaN,NaN,1,Консультант,1029256.0,NaN,NaN
2,FO2,Федеральный округ,СЕВЕРО-ЗАПАДНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,RF,NaN,NaN,13981992,NaN,NaN,NaN,1,Консультант,1216601.0,NaN,NaN
3,FO3,Федеральный округ,ЮЖНЫЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,RF,NaN,NaN,16466084,NaN,NaN,NaN,1,Консультант,1059500.0,NaN,NaN
4,FO4,Федеральный округ,СЕВЕРО-КАВКАЗСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ,RF,NaN,NaN,9930933,NaN,NaN,NaN,1,Консультант,389344.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21097,77633701,Межселенная территория,Межселенные территории Чукотского муниципально...,77633000,NaN,8.0,0,NaN,NaN,NaN,1,Консультант,NaN,NaN,NaN
21098,77701000,Городской округ,Анадырь,77000000,г Анадырь,8.0,16299,Городской округ Анадырь,NaN,NaN,1,Консультант,1957550.0,NaN,NaN
21099,77705000,Городской округ,Певек,77000000,г Певек,8.0,5480,Городской округ Певек,NaN,NaN,1,Консультант,1949883.0,NaN,NaN
21100,77710000,Городской округ,Провиденский,77000000,пгт Провидения,8.0,3550,Городской округ Провидения,NaN,NaN,1,Консультант,1949882.0,NaN,NaN


Теперь отсортируем нашу таблицу, т.к. нам нужны только те строки, в которых в столбце "Родитель" содержится 5 нулей:

Для этого столбцу "Родитель" присвоим тип строки (чтобы найти вхождения 5 нулей):

In [5]:
oktmo['Родитель'] = oktmo['Родитель'].astype('str')
oktmo['Родитель'][50]

'14625000'

Далее переприсвоим перееменную oktmo, ведь нам нужны именно эти 2383 строки:

In [6]:
oktmo = oktmo[oktmo['Родитель'].str.contains('00000')]
oktmo.shape

(2383, 15)

Теперь вернем тип int столбцу "Родитель":

In [7]:
oktmo['Родитель'] = pd.to_numeric(oktmo['Родитель'])

<ipython-input-7-f5aa8ef06cdb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oktmo['Родитель'] = pd.to_numeric(oktmo['Родитель'])


In [8]:
print(type(oktmo['Родитель'][10]))
oktmo['Родитель'][10]

<class 'numpy.int64'>


14000000

Теперь посмотрим на id (столбец "id территории") и определим, есть ли в нем пропуски:

In [9]:
oktmo['id территории'].isna().sum()

2

2 строчки. Что за территории?

In [10]:
oktmo[oktmo['id территории'].isna()]

,Код,Тип,Наименование,Родитель,Дополнительные данные,Том,Численность,Наименование.1,Код.1,Комментарий,Актуальность,Источник,id территории,0.988909426987061,Unnamed: 14
474,15750000,Городской округ,город Стародуб,15000000,г Стародуб,1.0,18156,Городской округ г. Стародуб,NaN,NaN,1,Консультант,NaN,NaN,NaN
2402,46765000,Городской округ,Рошаль,46000000,г Рошаль,1.0,20022,Городской округ Рошаль,NaN,NaN,1,Консультант,NaN,NaN,NaN


Удаляем эти строки, поскольку эти 2 города вошли в состав городских округов.

In [12]:
oktmo = oktmo.dropna(subset=['id территории'])
oktmo.shape

(2381, 15)

Перепроверим:

In [13]:
oktmo['id территории'].isna().sum()

0

Теперь присвоим id вид int, т.к. это должны быть целочисленные значения:

In [14]:
oktmo['id территории'] = oktmo['id территории'].astype('int')

In [15]:
oktmo['id территории']

10       1645362
35       1645363
46       1645356
59       1645372
74       1645355
          ...   
21090    1949884
21098    1957550
21099    1949883
21100    1949882
21101    1949881
Name: id территории, Length: 2381, dtype: int32

Супер.   
Соберем все id в один список, по которому будем итерироваться:

In [38]:
id_of_reg = oktmo['id территории'].to_list()
id_of_reg[:10]

[1645362,
 1645363,
 1645356,
 1645372,
 1645355,
 1645364,
 1645371,
 1645368,
 1645373,
 1645358]

Мы нашли координаты для прорабатывания нашего массива с поиском точек.

Теперь прогоним эти id через функцию и создадим новый датафрейм, в котором будут id, названия территорий и координаты территорий:

In [77]:
df_oktmo = pd.DataFrame()
df_oktmo['id территории'] = id_of_reg
df_oktmo.head(10)

,id территории
0,1645362
1,1645363
2,1645356
3,1645372
4,1645355
5,1645364
6,1645371
7,1645368
8,1645373
9,1645358


Создаем столбец с названиями территорий:

In [78]:
df_oktmo['name'] = df_oktmo['id территории'].map(newdata.set_index('OSM_ID')['NAME'])
df_oktmo

,id территории,name
0,1645362,Белгородский район
1,1645363,Борисовский район
2,1645356,Вейделевский район
3,1645372,Волоконовский район
4,1645355,Ивнянский район
...,...,...
2376,1949884,Чукотский район
2377,1957550,городской округ Анадырь
2378,1949883,городской округ Певек
2379,1949882,Провиденский городской округ


И последний столбец - с координатами территорий:

In [79]:
df_oktmo['coordinates'] = df_oktmo['id территории'].map(newdata.set_index('OSM_ID')['coordinates'])
df_oktmo

,id территории,name,coordinates
0,1645362,Белгородский район,"[[(36.0918, 50.44544), (36.08665, 50.44836), (..."
1,1645363,Борисовский район,"[[(35.906363144783036, 50.43796331719628), (35..."
2,1645356,Вейделевский район,"[[(38.332999996701, 50.065183980491256), (38.3..."
3,1645372,Волоконовский район,"[[(37.48999, 50.4554), (37.49102, 50.45746), (..."
4,1645355,Ивнянский район,"[[(35.9962048, 51.0700776), (35.9868185, 51.07..."
...,...,...,...
2376,1949884,Чукотский район,"[[(-174.5003079992036, 67.68745099976489), (-1..."
2377,1957550,городской округ Анадырь,"[[(177.5047365, 64.7159161), (177.5055401, 64...."
2378,1949883,городской округ Певек,"[[(176.11740557810205, 70.07692480819284), (17..."
2379,1949882,Провиденский городской округ,"[[(-171.62839333103605, 65.26644781326158), (-..."


Загоняем врейм в эксель:

In [81]:
# df_oktmo.to_excel('C:/Users/User/Documents/boundary_file/Новый ОКТМО с координатами.xlsx')

In [115]:
newdata[newdata['NAME'] == 'Белгородский район']['coordinates'].tolist()

[[[(36.0918, 50.44544),
   (36.08665, 50.44836),
   (36.0907742, 50.4488776),
   (36.1102185, 50.4526667),
   (36.1181806, 50.4576826),
   (36.1314228, 50.4638184),
   (36.1312552, 50.4644053),
   (36.1423183, 50.4696867),
   (36.1494174, 50.4740133),
   (36.1644446, 50.4832344),
   (36.1755077, 50.4872873),
   (36.1702276, 50.4984309),
   (36.1679646, 50.5037086),
   (36.1470956, 50.5015763),
   (36.1413964, 50.5001369),
   (36.1413577, 50.5003253),
   (36.1397401, 50.5081978),
   (36.1394687, 50.5095186),
   (36.1367029, 50.5161806),
   (36.1366823, 50.5163528),
   (36.1354458, 50.5266781),
   (36.1289923, 50.5271576),
   (36.128657, 50.5291822),
   (36.1216168, 50.5300879),
   (36.122455, 50.5388778),
   (36.1306954, 50.5430559),
   (36.1313507, 50.5429685),
   (36.1319296, 50.5429022),
   (36.1320763, 50.5429088),
   (36.1320725, 50.5429333),
   (36.1320964, 50.5429754),
   (36.1321147, 50.5430565),
   (36.1321367, 50.5431429),
   (36.1320606, 50.5431364),
   (36.1316636, 50.543182

In [116]:
df_oktmo[df_oktmo['name'] == 'Белгородский район']['coordinates'].tolist()

[[[(36.0918, 50.44544),
   (36.08665, 50.44836),
   (36.0907742, 50.4488776),
   (36.1102185, 50.4526667),
   (36.1181806, 50.4576826),
   (36.1314228, 50.4638184),
   (36.1312552, 50.4644053),
   (36.1423183, 50.4696867),
   (36.1494174, 50.4740133),
   (36.1644446, 50.4832344),
   (36.1755077, 50.4872873),
   (36.1702276, 50.4984309),
   (36.1679646, 50.5037086),
   (36.1470956, 50.5015763),
   (36.1413964, 50.5001369),
   (36.1413577, 50.5003253),
   (36.1397401, 50.5081978),
   (36.1394687, 50.5095186),
   (36.1367029, 50.5161806),
   (36.1366823, 50.5163528),
   (36.1354458, 50.5266781),
   (36.1289923, 50.5271576),
   (36.128657, 50.5291822),
   (36.1216168, 50.5300879),
   (36.122455, 50.5388778),
   (36.1306954, 50.5430559),
   (36.1313507, 50.5429685),
   (36.1319296, 50.5429022),
   (36.1320763, 50.5429088),
   (36.1320725, 50.5429333),
   (36.1320964, 50.5429754),
   (36.1321147, 50.5430565),
   (36.1321367, 50.5431429),
   (36.1320606, 50.5431364),
   (36.1316636, 50.543182

### Шаг 2. Работа с точками.

Откроем теперь файл с примерами точек:

In [17]:
exa_points = pd.read_excel('C:/Users/User/Downloads/Telegram Desktop/Примеры точек в полигоне.xlsx')
exa_points

,Точка,Где находится,Попададает/ не попадает в полигон
0,"62.865671, 51.900399",пос Синдор,нет
1,"62.598375, 50.886569",гор Емва,нет
2,"63.306072, 51.780135",оз Тойюты,да
3,"62.922938, 52.499906",оз Безымянное,да
4,"63.250082, 52.639075",Пустырь,да


~~Напишем функцию, принимающую на вход таблицу exa_points и название столбца с координатами. Функция будет разбивать столбец с координатами на 2 части и формировать новые 2 столбца: "Долгота" и "Широта". Далее новым столбцам будет присвоен тип float и на выходе из функции мы будем получать готовую таблицу со значениями координат точек.~~

In [18]:
# def longitude_latitude(df, column):
#     df['Долгота'] = df["{}".format(column)].str.split(',').str[1].astype('float')
#     df['Широта'] = df["{}".format(column)].str.split(',').str[0].astype('float')
#     return df['Долгота'], df['Широта']
# exa_points['Долгота'], exa_points['Широта'] = longitude_latitude(exa_points, 'Точка')
# exa_points

~~Для удобства поменяем местами столбцы:~~

In [19]:
# exa_points = exa_points[['Точка', 'Долгота', 'Широта', 'Где находится', 'Попададает/ не попадает в полигон']]
# exa_points

Напишем функцию, принимающую на вход строчки из таблицы exa_points. Функция будет разбивать столбец с координатами на 2 части (по запятой) и присваивать тип float получившимся координатам. на выходе из функции мы будем получать готовую таблицу со значениями координат точек.

In [20]:
def string_to_float(row):
    point = row['Точка']     # идентифицируем нужную точку
    point = re.split(r'[,\s]+', point)     # при помощи регулярных выражений разделим значения по запятой
    new_point = [float(x) for x in point]   # переведем каждое значение в формат чисел
    return new_point

In [21]:
exa_points['Точка в числовом формате'] = exa_points.apply(string_to_float, axis=1)

In [22]:
exa_points

,Точка,Где находится,Попададает/ не попадает в полигон,Точка в числовом формате
0,"62.865671, 51.900399",пос Синдор,нет,"[62.865671, 51.900399]"
1,"62.598375, 50.886569",гор Емва,нет,"[62.598375, 50.886569]"
2,"63.306072, 51.780135",оз Тойюты,да,"[63.306072, 51.780135]"
3,"62.922938, 52.499906",оз Безымянное,да,"[62.922938, 52.499906]"
4,"63.250082, 52.639075",Пустырь,да,"[63.250082, 52.639075]"


Проверим возможность итерации по столбцу 'Точка в числовом формате':

In [23]:
exa_points['Точка в числовом формате'][0][1]

51.900399

Супер! Теперь напишем код для идентификации точек внутри полигона или снаружи.

### Шаг 3. Проверка вхождения точки в полигон.

Проверим вхождения точек в полигон:

In [191]:
# def find_point(x, y, poly, include_edges=True):
#     '''
#     Test if point (x,y) is inside polygon poly.


#     poly is N-vertices polygon defined as 
#     [(x1,y1),...,(xN,yN)] or [(x1,y1),...,(xN,yN),(x1,y1)]
#     (function works fine in both cases)



#     Geometrical idea: point is inside polygon if horisontal beam
#     to the right from point crosses polygon even number of times. 
#     Works fine for non-convex polygons.
#     '''
#     n = len(poly)
#     inside = False    
#     p1x, p1y = poly[0]     # получаем координаты х и у

#     for i in range(1, n + 1):
#         p2x, p2y = poly[i % n]   # учитываем вероятность отклонения

#         if p1y == p2y:        
#             if y == p1y:
#                 if min(p1x, p2x) <= x <= max(p1x, p2x):
#                     # точка находится на границе полигона
#                     inside = include_edges
#                     break
#                 elif x < min(p1x, p2x):  # точка снаружи полигона
#                     inside = not inside
#         else:  # p1y!= p2y
#             if min(p1y, p2y) <= y <= max(p1y, p2y):
#                 xinters = (y - p1y) * (p2x - p1x) / float(p2y - p1y) + p1x


#                 if x == xinters:  # если точка на границе полигона
#                     inside = include_edges
#                     break



#                 if x < xinters:  # точка снаружи полигона
#                     inside = not inside



#         p1x, p1y = p2x, p2y

#     return inside

In [286]:
# def split_to_subpolygons(latitude, longitude, polygon, name_of_poly):
#     list_of_answers = []
#     for value in range(len(polygon)):
#         subpolygon = polygon[value]
#         if len(subpolygon[0]) == 2:
#             list_of_answers.append(find_point(latitude, longitude, subpolygon)) 
#         else:
#             list_of_answers.append(find_point(latitude, longitude, subpolygon[0]))
#     if any(list_of_answers):
#         return "Точка внутри полигона", name_of_poly
#     return "Точка вне полигона", ''

In [287]:
# split_to_subpolygons(x, y, coord_of_poly('Б87608000')[0], coord_of_poly('Б87608000')[1])

[(50.5456434, 62.3112302),
 (50.3741642, 62.3253479),
 (50.4029418, 62.3940109),
 (50.2851099, 62.4051184),
 (50.2930091, 62.4141267),
 (50.3025663, 62.4396585),
 (50.3956803, 62.4281593),
 (50.4126101, 62.4657996),
 (50.5988851, 62.4467557),
 (50.5806743, 62.3957566),
 (50.584885, 62.3952474),
 (50.5456434, 62.3112302)]

In [188]:
'''Код в данной ячейке написан для введения координат вручную (если это когда-нибудь понадобится). Использовала для теста'''
# x = float(input('Введите долготу: '))
# y = float(input('Введите широту: '))
# split_to_subpolygons(x, y, coordinates_0)

'Код в данной ячейке написан для введения координат вручную (если это когда-нибудь понадобится). Использовала для теста'

Для начала тестирования напишем код, чтобы проверить наш скрипт на межселенной территории Б87608000:

Также создадим 3 новых столбца в таблице exa_points, где отметим получившиеся результаты, вызвав все написанные функции:

In [196]:
# lst_point = []
# lst_poly = []
# lst_id = []

In [197]:
# for i in exa_points['Точка в числовом формате']:
#     x = i[1]
#     y = i[0]
#     lst_point.append(split_to_subpolygons(x, y, coord_of_poly('Б87608000')[0], coord_of_poly('Б87608000')[1])[0])
#     lst_poly.append(split_to_subpolygons(x, y, coord_of_poly('Б87608000')[0], coord_of_poly('Б87608000')[1])[1])
#     lst_id.append('Б87608000')
# exa_points['Положение точки']  = lst_point
# exa_points['Название полигона, куда попала точка'] = lst_poly
# exa_points['id территории'] = lst_id
# exa_points

,Точка,Где находится,Попададает/ не попадает в полигон,Точка в числовом формате,Положение точки,"Название полигона, куда попала точка",id территории
0,"62.865671, 51.900399",пос Синдор,нет,"[62.865671, 51.900399]",Точка вне полигона,,Б87608000
1,"62.598375, 50.886569",гор Емва,нет,"[62.598375, 50.886569]",Точка вне полигона,,Б87608000
2,"63.306072, 51.780135",оз Тойюты,да,"[63.306072, 51.780135]",Точка внутри полигона,"Межселенные территории, входящие в состав терр...",Б87608000
3,"62.922938, 52.499906",оз Безымянное,да,"[62.922938, 52.499906]",Точка внутри полигона,"Межселенные территории, входящие в состав терр...",Б87608000
4,"63.250082, 52.639075",Пустырь,да,"[63.250082, 52.639075]",Точка внутри полигона,"Межселенные территории, входящие в состав терр...",Б87608000


In [371]:
exa_points['Название полигона, куда попала точка'][2]

'Межселенные территории, входящие в состав территории Княжпогостского района'

Далее скорректируем нашу функцию: раньше я передавала только один полигон. Теперь же я передам все. Если точка будет входить в какой-то полигон, мы получим подтверждение в виде "Точка внутри полигона", далее название полигона и его id.

Для начала откроем новый файл с названиями территорий и их координатами:

In [123]:
# df_oktmo[df_oktmo['name'] == 'Белгородский район']['coordinates'].tolist()

Отсюда мы будем брать координаты и названия регионов.

In [122]:
coord_all = df_oktmo['coordinates'].tolist()
print(len(coord_all))

2381


In [125]:
name_all = df_oktmo['name'].tolist()

Теперь перепишем функцию подготовки территорий:

In [356]:
def find_point(x, y, poly, include_edges=True):
    '''
    Test if point (x,y) is inside polygon poly.


    poly is N-vertices polygon defined as 
    [(x1,y1),...,(xN,yN)] or [(x1,y1),...,(xN,yN),(x1,y1)]
    (function works fine in both cases)



    Geometrical idea: point is inside polygon if horisontal beam
    to the right from point crosses polygon even number of times. 
    Works fine for non-convex polygons.
    '''
    n = len(poly)
    inside = False    
    
    p1x, p1y = poly[0]     # получаем координаты х и у

    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]   # учитываем вероятность отклонения

        if p1y == p2y:        
            if y == p1y:
                if min(p1x, p2x) <= x <= max(p1x, p2x):
                    # точка находится на границе полигона
                    inside = include_edges
                    break
                elif x < min(p1x, p2x):  # точка снаружи полигона
                    inside = not inside
        else:  # p1y!= p2y
            if min(p1y, p2y) <= y <= max(p1y, p2y):
                xinters = (y - p1y) * (p2x - p1x) / float(p2y - p1y) + p1x


                if x == xinters:  # если точка на границе полигона
                    inside = include_edges
                    break



                if x < xinters:  # точка снаружи полигона
                    inside = not inside

    p1x, p1y = p2x, p2y

    return inside

In [357]:
def split_to_subpolygons(latitude, longitude, polygon, name_of_poly):
    list_of_answers = []
    for value in range(len(polygon)):
        subpolygon = polygon[value]
        if len(subpolygon[0]) == 2:
            list_of_answers.append(find_point(latitude, longitude, subpolygon)) 
        else:
            list_of_answers.append(find_point(latitude, longitude, subpolygon[0]))
    return list_of_answers
    if any(list_of_answers):
        return "Точка внутри полигона", name_of_poly[value]
    return "Точка вне полигона", ''

In [329]:
# def polygons_to_polygon(latitude, longitude, polygons, name_of_poly):
#     for polygon in range(len(polygons)):
#         print(polygons[polygon])      # координаты полигона
#         print(polygon)                # индекс строки датафейма
#         print(name_of_poly[polygon])  # имя территории по индексу полигона
#         break

In [385]:
def polygons_to_polygon(latitude, longitude, polygons, name_of_poly):
    lst_of_true = []
    for polygon in range(len(polygons)):
        a = split_to_subpolygons(latitude, longitude, polygons[polygon], name_of_poly)
        lst_of_true.append(a)
    return lst_of_true
#     return sum(lst_of_true, [])
#         for i in range(len(lst_of_true)):
#             if s[i] == 'True':
#                 print("Да", i)
#             else:
#                 continue
    

In [386]:
63.306072, 51.780135

(63.306072, 51.780135)

In [387]:
f = polygons_to_polygon(51.780135, 63.306072, df_oktmo['coordinates'], df_oktmo['name'])

In [388]:
len(f)

2381

In [389]:
for i in range(len(f)):
    if f[i]

[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False]
[False, False, False, False, False, False, False, False, False, False]
[False]
[False]
[False]
[False]
[False]
[False, False]
[False, False]
[False]
[False, False]
[False]
[False, False]
[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False, False, False]
[False]
[False, False]
[False]
[False, False, False, False, False]
[False]
[False]
[False]
[False, False, False, False, False, False]
[False]
[False, False]
[False, False]
[False]
[False]
[False]
[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False, False, False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[Fals

In [402]:
f

[[False, False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False, False],
 [False, False, False, False, False, False, False, False, False, False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False, False],
 [False, False],
 [False],
 [False, False],
 [False],
 [False, False],
 [False, False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False, False, False, False],
 [False],
 [False, False],
 [False],
 [False, False, False, False, False],
 [False],
 [False],
 [False],
 [False, False, False, False, False, False],
 [False],
 [False, False],
 [False, False],
 [False],
 [False],
 [False],
 [False, False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False],
 [False, False, False, False, False],
 [False],
 [False],
 [False],
 [False]

In [349]:
s = ['False', 'False', 'False', 'False', 'True', 'False', 'False']

In [400]:
for i in range(len(f)):
    print(f[i])
#         if 'True' in j:
#             print("Да", i)
#         else:
#             print('No')

[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False]
[False, False, False, False, False, False, False, False, False, False]
[False]
[False]
[False]
[False]
[False]
[False, False]
[False, False]
[False]
[False, False]
[False]
[False, False]
[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False, False, False]
[False]
[False, False]
[False]
[False, False, False, False, False]
[False]
[False]
[False]
[False, False, False, False, False, False]
[False]
[False, False]
[False, False]
[False]
[False]
[False]
[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False, False, False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False, False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[Fals

In [382]:
set(f)

{False, True}

In [401]:
[sample for sample in f if any(sample)]

[[True],
 [True],
 [True],
 [True],
 [True, False],
 [True],
 [True, False, False, False, False],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True],
 [True, False]]

In [372]:
df_oktmo[df_oktmo['name'] == 'Княжпогостский район']

,id территории,name,coordinates
537,1082939,Княжпогостский район,"[[(50.6089698, 62.2637749), (50.6136092, 62.26..."


In [395]:
next(x for x in f if "True" in x)

StopIteration: 